In [2]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [4]:
client.create_experiment(name="my-cool-experiment")

'2'

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: ea0b5d61e0d14ef3973d8dc436e2c061, rmse: 6.3250
run id: 6478c6eb23e8489ebec3463ff42ef91d, rmse: 6.3250
run id: 490c43a5280748468e1ac98d6e97d13b, rmse: 6.3369
run id: 793eeef8d8ba4680b4c8987fff482f9f, rmse: 6.3779
run id: 6e286c0784bf448ea2ab6a054f0a3565, rmse: 6.3806


In [7]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [10]:
run_id="6478c6eb23e8489ebec3463ff42ef91d"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1723259063431, current_stage='None', description=None, last_updated_timestamp=1723259063431, name='nyc-taxi-regressor', run_id='6478c6eb23e8489ebec3463ff42ef91d', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/6478c6eb23e8489ebec3463ff42ef91d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [14]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage:{version.current_stage}")

version: 3, stage:None


/tmp/ipykernel_14838/3475322982.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [30]:
model_version=1
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_14838/2796152230.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1723257956883, current_stage='Production', description='', last_updated_timestamp=1723260290270, name='nyc-taxi-regressor', run_id='ea0b5d61e0d14ef3973d8dc436e2c061', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/ea0b5d61e0d14ef3973d8dc436e2c061/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>

In [31]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1723257956883, current_stage='Production', description='The model version 1 was transitioned to Production on 2024-08-10', last_updated_timestamp=1723260291088, name='nyc-taxi-regressor', run_id='ea0b5d61e0d14ef3973d8dc436e2c061', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/ea0b5d61e0d14ef3973d8dc436e2c061/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>

In [20]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [21]:
df = read_dataframe("./data/green_tripdata_2021-01.parquet")

In [24]:
client.download_artifacts(run_id=run_id, path="preprocessor", dst_path='.')

'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [25]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [26]:
X_test = preprocess(df, dv)

In [27]:
target = "duration"
y_test = df[target].values

In [34]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


CPU times: user 6.03 s, sys: 7.82 ms, total: 6.04 s
Wall time: 3.32 s


{'rmse': 4.860989678366851}